In [1]:

import os, time, pandas as pd,random
from datetime import datetime
import json
import pandas as pd

from sklearn.linear_model import LinearRegression

# Output directory
output_dir = "sensor_data/json"
os.makedirs(output_dir, exist_ok=True)

# Define the headers for the CSV file related to customer transactions
header = ['sensor_id', 'timestamp', 'temperature', 'humidity']


In [2]:

# Generate random temperature and humidity data
def generate_temperature_humidity_data(sensor_id=1):
    temperature = round(random.uniform(18, 30), 2)
    return {
        "sensor_id": sensor_id,
        "timestamp": datetime.now().isoformat(),
        "temperature": temperature,
        "temperature_f": round(temperature * 9 / 5 + 32, 2),
        "humidity": round(random.uniform(40, 70), 2) if random.random() > 0.2 else None,
        "motion_detected": int(round(random.uniform(0,1),0))
    }


In [3]:
# Save data as CSV
def save_temperature_humidity_csv(filename, data):
    filepath = os.path.join(output_dir, filename)
    df = pd.DataFrame(data)
    df.to_csv(filepath, mode='w', index=False, header=True)  
    print(f"Saved Temperature & Humidity CSV: {filepath}")

In [6]:

# User input for number of times to run
num_iterations = int(input("Enter number of readings to generate: "))

temperature_humidity_sensor_data = []
# Generate and save sensor data
for i in range(num_iterations):
    temperature_humidity_data = generate_temperature_humidity_data()
    temperature_humidity_sensor_data.append(temperature_humidity_data)
    print(temperature_humidity_data)
    # time.sleep(1)

    # Adda a duplicate a t every 3rd reading
    if i % 3 == 0 and i != 0:
        temperature_humidity_sensor_data.append(temperature_humidity_data)
        print(temperature_humidity_data)

save_temperature_humidity_csv("temperature_humidity.csv", temperature_humidity_sensor_data)

filename = "temperature_humidity_data.json"
filepath = os.path.join(output_dir, filename)
try:
    filedata = json.load(open(filename))
    filedata.append(temperature_humidity_data)
except:
    filedata = []

with open(filepath, mode='w') as json_file:
    # writer = csv.writer(file)
    # json_file.write('[')
    # json.(temperature_humidity_data, json_file, indent=4)
    json.dump(filedata, json_file, indent=4)
    # json_file.write(']')

    # Write all data at once
    #writer.writerows(data)
json_file.close
# Close the file 
print(f'JSON data written to {filename}')

Enter number of readings to generate:  50


{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.473980', 'temperature': 29.59, 'temperature_f': 85.26, 'humidity': 46.27, 'motion_detected': 1}
{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.474146', 'temperature': 19.37, 'temperature_f': 66.87, 'humidity': 49.63, 'motion_detected': 1}
{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.474196', 'temperature': 25.16, 'temperature_f': 77.29, 'humidity': 62.82, 'motion_detected': 1}
{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.474235', 'temperature': 21.05, 'temperature_f': 69.89, 'humidity': None, 'motion_detected': 1}
{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.474235', 'temperature': 21.05, 'temperature_f': 69.89, 'humidity': None, 'motion_detected': 1}
{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.474290', 'temperature': 21.66, 'temperature_f': 70.99, 'humidity': 53.77, 'motion_detected': 0}
{'sensor_id': 1, 'timestamp': '2025-03-31T15:31:24.474325', 'temperature': 22.52, 'temperature_f': 72.54, 'humidity': 40

In [7]:
df = pd.DataFrame(temperature_humidity_sensor_data)

save_temperature_humidity_csv("temperature_humidity_errors.csv", df)

df = df.drop_duplicates()

save_temperature_humidity_csv("temperature_humidity.csv", df)

Saved Temperature & Humidity CSV: sensor_data/json/temperature_humidity_errors.csv
Saved Temperature & Humidity CSV: sensor_data/json/temperature_humidity.csv


In [8]:
# missing_values = df.isnull().sum()

# if missing_values.sum() > 0:
#     missing_columns = missing_values[missing_values > 0].index.tolist()
#     print("\nColumns with missing values:", missing_columns)
#     df = df.dropna()
#     print("\nDropped rows with missing values.")

In [12]:
def impute_missing_values(df, strategy='mean'): 
    if strategy == 'mean':
        df['humidity'] = df['humidity'].fillna(df['humidity'].mean()).round()
    elif strategy == 'median':
        df['humidity'] = df['humidity'].fillna(df['humidity'].median()).round()
    elif strategy == 'mode':
        df['humidity'] = df['humidity'].fillna(df['humidity'].mode()).round()
    elif strategy == 'regression':
        hum_train = df.dropna(subset=['humidity'])
        model = LinearRegression()
        model.fit(hum_train[['temperature']], hum_train['humidity'])
        hum_missing = df[df['humidity'].isnull()]
        df.loc[df['humidity'].isnull(), 'humidity'] = model.predict(hum_missing[['temperature']])
    return df

In [13]:
df = impute_missing_values(df, strategy='median')

In [14]:
save_temperature_humidity_csv("temperature_humidity.csv", df)

Saved Temperature & Humidity CSV: sensor_data/json/temperature_humidity.csv
